# bruh

In [5]:
!git config --global user.name "Belaleatsbanana"
!git config --global user.email "belalofficial123@gmail.com"

In [ ]:
!git clone https://github.com/Belaleatsbanana/ml2.git

Cloning into 'ml2'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 47.04 KiB | 1.62 MiB/s, done.
/content/ml2/ml2


In [8]:
%pip install -r requirements.txt